# AI-Based Candidate Screening System

In [ ]:

# Required Libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import openai

# Set up OpenAI API (Make sure to set your API key here)
openai.api_key = 'your-api-key'
    

### Expanded Dataset of Candidates and Questions

In [ ]:

# Expanded mock interview data (responses for 5 questions)
candidates = {
    'Candidate A': [
        "Supervised learning uses labeled data, while unsupervised learning uses unlabeled data. Examples include classification for supervised and clustering for unsupervised learning.",
        "To handle imbalanced data, I would use SMOTE, oversampling, and metrics like precision, recall, and F1-score.",
        "I built a recommendation system using collaborative filtering and matrix factorization to solve data sparsity issues.",
        "Overfitting can be reduced using techniques like cross-validation, early stopping, regularization, and dropout in neural networks.",
        "In the AI landscape, reinforcement learning stands out for its ability to learn through trial and error using feedback, commonly seen in game playing and robotics."
    ],
    'Candidate B': [
        "Supervised learning is when data is labeled, and unsupervised is when it’s not.",
        "I would balance the dataset by using different sampling methods and focusing on precision.",
        "I built a chatbot using NLP, but I had some challenges with accuracy.",
        "Overfitting can be managed by reducing model complexity and tuning hyperparameters.",
        "Reinforcement learning learns from feedback and is useful for specific tasks like games."
    ],
    'Candidate C': [
        "Supervised learning has labels, unsupervised doesn't.",
        "I’d change the dataset or use a different model to fix imbalance issues.",
        "I worked on some AI-related projects, but I don't remember the details.",
        "Overfitting can be handled by tweaking some parameters.",
        "Reinforcement learning is important, but I haven't worked with it much."
    ]
}

# Expanded predefined expert answers for comparison (gold standard)
gold_standard = {
    'Question 1': "Supervised learning uses labeled data to train models and predict outcomes. Unsupervised learning finds hidden patterns in unlabeled data, such as clustering or dimensionality reduction.",
    'Question 2': "To handle imbalanced data, you can use oversampling, undersampling, SMOTE, or cost-sensitive algorithms. It's important to use evaluation metrics like precision, recall, and F1-score to assess model performance.",
    'Question 3': "In my recent project, I built a recommendation system using collaborative filtering. The challenge was sparse data, which I addressed using matrix factorization and regularization techniques.",
    'Question 4': "To avoid overfitting, I used regularization techniques like L2 regularization, early stopping, and cross-validation. In deep learning, dropout is a common technique.",
    'Question 5': "Reinforcement learning is a type of machine learning where agents learn through trial and error using rewards. It's applied in game AI and robotics for decision-making in dynamic environments."
}
    

### Function to Evaluate Candidates using GPT Model

In [ ]:

# Function to evaluate responses using OpenAI GPT model
def gpt_evaluate(candidate_response, question, gold_standard):
    prompt = f'''
    You are an AI interview evaluator. The candidate's answer is below, followed by the correct expert answer. Rate the candidate's answer on a scale of 0 to 10 based on accuracy, depth of understanding, and clarity.

    Question: {question}
    
    Candidate's answer: {candidate_response}

    Expert answer: {gold_standard}

    Rate the candidate's answer (0 to 10) and provide a brief explanation:
    '''
    
    response = openai.Completion.create(
      engine="gpt-4",  # You can use 'gpt-3.5-turbo' or another model if available
      prompt=prompt,
      max_tokens=150,
      temperature=0
    )
    
    # Parse the GPT response for the score
    return response.choices[0].text.strip()

# Evaluate each candidate's response using GPT
gpt_scores = {}
for candidate, responses in candidates.items():
    gpt_scores[candidate] = []
    for i, response in enumerate(responses):
        question = f"Question {i+1}"
        expert_answer = gold_standard[question]
        score = gpt_evaluate(response, question, expert_answer)
        gpt_scores[candidate].append(score)

# Display GPT scores for each candidate
for candidate, scores in gpt_scores.items():
    print(f"{candidate}'s scores (as evaluated by GPT): {scores}")
    

### Ranking Candidates

In [ ]:

# Aggregating total score for each candidate
total_gpt_scores = {candidate: np.mean([float(score.split()[0]) for score in scores]) for candidate, scores in gpt_scores.items()}

# Ranking candidates based on total scores
ranked_candidates = sorted(total_gpt_scores.items(), key=lambda item: item[1], reverse=True)

# Display ranked candidates
print("\nRanked Candidates (Total Score out of 10):")
for rank, (candidate, score) in enumerate(ranked_candidates, start=1):
    print(f"Rank {rank}: {candidate} with score {score:.2f}/10")
    